## Introduction to the Keras Tuner


**Overview**

**The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called hyperparameter tuning or hypertuning.**

**Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:**

**Model hyperparameters which influence model selection such as the number and width of hidden layers
Algorithm hyperparameters which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier.**

In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [2]:
# Load the MNIST dataset
(img_train, label_train), (img_test, label_test) = keras.datasets.mnist.load_data()

In [3]:
print(img_train.shape)
print(label_train.shape)
print(img_test.shape)
print(label_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [6]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])

    return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [8]:
# Hyperband algorithm should be used with EarlyStopping
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 40s]
val_accuracy: 0.9779999852180481

Best val_accuracy So Far: 0.9799166917800903
Total elapsed time: 00h 10m 05s
INFO:tensorflow:Oracle triggered exit


In [10]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.001.



In [21]:
best_hps.get_config()

{'space': [{'class_name': 'Int',
   'config': {'name': 'units',
    'default': None,
    'conditions': [],
    'min_value': 32,
    'max_value': 512,
    'step': 32,
    'sampling': 'linear'}},
  {'class_name': 'Choice',
   'config': {'name': 'learning_rate',
    'default': 0.01,
    'conditions': [],
    'values': [0.01, 0.001, 0.0001],
    'ordered': True}}],
 'values': {'units': 512,
  'learning_rate': 0.001,
  'tuner/epochs': 10,
  'tuner/initial_epoch': 4,
  'tuner/bracket': 2,
  'tuner/round': 2,
  'tuner/trial_id': '0013'}}

In [22]:
best_hps.values

{'units': 512,
 'learning_rate': 0.001,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0013'}

In [18]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=10, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2387 - accuracy: 0.9295 - val_loss: 0.1236 - val_accuracy: 0.9647
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0982 - accuracy: 0.9707 - val_loss: 0.1011 - val_accuracy: 0.9718
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0657 - accuracy: 0.9796 - val_loss: 0.0876 - val_accuracy: 0.9733
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0448 - accuracy: 0.9861 - val_loss: 0.0851 - val_accuracy: 0.9761
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0320 - accuracy: 0.9900 - val_loss: 0.0821 - val_accuracy: 0.9767
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0242 - accuracy: 0.9924 - val_loss: 0.0799 - val_accuracy: 0.9779
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0192 - accuracy: 0.9940 - val_loss: 0.0853 - val_accuracy:

In [19]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
history2 = hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/38
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2405 - accuracy: 0.9312 - val_loss: 0.1333 - val_accuracy: 0.9621
Epoch 2/38
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0998 - accuracy: 0.9704 - val_loss: 0.0956 - val_accuracy: 0.9719
Epoch 3/38
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0644 - accuracy: 0.9803 - val_loss: 0.0882 - val_accuracy: 0.9749
Epoch 4/38
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0446 - accuracy: 0.9862 - val_loss: 0.0845 - val_accuracy: 0.9753
Epoch 5/38
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0341 - accuracy: 0.9896 - val_loss: 0.0854 - val_accuracy: 0.9753
Epoch 6/38
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0252 - accuracy: 0.9920 - val_loss: 0.0918 - val_accuracy: 0.9747
Epoch 7/38
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0201 - accuracy: 0.9934 - val_loss: 0.0913 - val_accuracy: